In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image
import os
import time

# create directory for save result_img

In [2]:
if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')

# define function to create thumbnail

In [3]:
def to_img(x):
    out = 0.5 * (x + 1)
    out = out.clamp(0, 1)
    out = out.view(-1, 1, 28, 28)
    return out

# initialization

In [5]:
batch_size = 128
num_epoch = 100
z_dimension = 100  # noise dimension

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
# mnist = datasets.MNIST('./mnist_data', transform=img_transform)
dataloader = DataLoader( datasets.MNIST("./mnist_data", train=True, download=True,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
#                               transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=batch_size, shuffle=True,
    num_workers=4, pin_memory=True)

# define models

In [7]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 5, padding=2),  # batch, 32, 28, 28
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=2),  # batch, 32, 14, 14
            )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 5, padding=2),  # batch, 64, 14, 14
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=2)  # batch, 64, 7, 7
        )
        self.fc = nn.Sequential(
            nn.Linear(64*7*7, 1024),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
 
    def forward(self, x):
        '''
        x: batch, width, height, channel=1
        '''
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [8]:
class generator(nn.Module):
    def __init__(self, input_size, num_feature):
        super(generator, self).__init__()
        self.fc = nn.Linear(input_size, num_feature)  # batch, 3136=1x56x56
        self.br = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.ReLU(True)
        )
        self.downsample1 = nn.Sequential(
            nn.Conv2d(1, 50, 3, stride=1, padding=1),  # batch, 50, 56, 56
            nn.BatchNorm2d(50),
            nn.ReLU(True)
        )
        self.downsample2 = nn.Sequential(
            nn.Conv2d(50, 25, 3, stride=1, padding=1),  # batch, 25, 56, 56
            nn.BatchNorm2d(25),
            nn.ReLU(True)
        )
        self.downsample3 = nn.Sequential(
            nn.Conv2d(25, 1, 2, stride=2),  # batch, 1, 28, 28
            nn.Tanh()
        )
 
    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), 1, 56, 56)
        x = self.br(x)
        x = self.downsample1(x)
        x = self.downsample2(x)
        x = self.downsample3(x)
        return x

In [9]:
D = discriminator().to(device)  # discriminator model
G = generator(z_dimension, 3136).to(device)  # generator model

# define optimizer

In [10]:
criterion = nn.BCELoss()  # binary cross entropy
 
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

# start training

In [11]:
for epoch in range(num_epoch):
    since = time.time()
    for i, (data, target) in enumerate(dataloader):
        num_img = data.size(0)
        data, target = data.to(device), target.to(device)
        
        real_label = torch.ones(num_img, 1).to(device)
        fake_label = torch.zeros(num_img, 1).to(device)

        pred_real = D(data)
        d_loss_real = criterion(pred_real, real_label)
        real_scores = pred_real  # closer to 1 means better
 
        # compute loss of fake_img
        z = torch.randn(num_img, z_dimension).to(device)
        fake_data = G(z)
        pred_fake = D(fake_data)
        d_loss_fake = criterion(pred_fake, fake_label)
        fake_scores = pred_fake  # closer to 0 means better
 
        # bp and optimize
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
 
        # ===============train generator
        # compute loss of fake_img
        z = torch.randn(num_img, z_dimension).to(device)
        fake_img = G(z)
        output = D(fake_img)
        g_loss = criterion(output, real_label)
 
        # bp and optimize
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
 
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], d_loss: {:.6f}, g_loss: {:.6f} '
                  'D real: {:.6f}, D fake: {:.6f}'
                  .format(epoch, num_epoch, d_loss.item(), g_loss.item(),
                          real_scores.data.mean(), fake_scores.data.mean()))
    print('Epoch[{}/100], time_consumed:{:.6f}'.format(epoch,time.time()-since))
    if epoch == 0:
        real_images = to_img(data.cpu().data)
        save_image(real_images, './dc_img/real_images.png')
    
    fake_images = to_img(fake_data.cpu().data)
    save_image(fake_images, './dc_img/fake_images-{}.png'.format(epoch+1))

Epoch [0/100], d_loss: 0.254484, g_loss: 2.658250 D real: 0.906481, D fake: 0.105014
Epoch [0/100], d_loss: 0.096534, g_loss: 3.189066 D real: 0.943908, D fake: 0.027776
Epoch [0/100], d_loss: 0.420174, g_loss: 3.100069 D real: 0.848257, D fake: 0.079768
Epoch [0/100], d_loss: 0.488493, g_loss: 2.563075 D real: 0.895435, D fake: 0.238030
Epoch[0/100], time_consumed:37.429294
Epoch [1/100], d_loss: 0.265498, g_loss: 2.712176 D real: 0.863886, D fake: 0.062642
Epoch [1/100], d_loss: 0.268767, g_loss: 3.876229 D real: 0.906216, D fake: 0.105271
Epoch [1/100], d_loss: 0.243188, g_loss: 3.524614 D real: 0.926796, D fake: 0.104459
Epoch [1/100], d_loss: 0.285698, g_loss: 5.132143 D real: 0.840146, D fake: 0.034290
Epoch[1/100], time_consumed:36.987188
Epoch [2/100], d_loss: 0.214583, g_loss: 4.612216 D real: 0.921559, D fake: 0.088741
Epoch [2/100], d_loss: 0.209107, g_loss: 2.920372 D real: 0.946017, D fake: 0.100920
Epoch [2/100], d_loss: 0.253552, g_loss: 3.554862 D real: 0.971049, D fake

Epoch [21/100], d_loss: 0.419677, g_loss: 2.205489 D real: 0.886316, D fake: 0.169886
Epoch[21/100], time_consumed:36.631048
Epoch [22/100], d_loss: 0.356339, g_loss: 2.956194 D real: 0.877093, D fake: 0.110754
Epoch [22/100], d_loss: 0.367204, g_loss: 3.125468 D real: 0.851753, D fake: 0.062183
Epoch [22/100], d_loss: 0.528007, g_loss: 2.048313 D real: 0.780637, D fake: 0.060386
Epoch [22/100], d_loss: 0.460782, g_loss: 2.901793 D real: 0.867405, D fake: 0.153013
Epoch[22/100], time_consumed:36.350110
Epoch [23/100], d_loss: 0.358077, g_loss: 2.157612 D real: 0.907487, D fake: 0.171781
Epoch [23/100], d_loss: 0.520742, g_loss: 4.701721 D real: 0.805330, D fake: 0.016311
Epoch [23/100], d_loss: 0.468362, g_loss: 1.916243 D real: 0.834300, D fake: 0.124882
Epoch [23/100], d_loss: 0.519790, g_loss: 4.098829 D real: 0.812545, D fake: 0.039730
Epoch[23/100], time_consumed:36.421290
Epoch [24/100], d_loss: 0.418348, g_loss: 1.963414 D real: 0.876911, D fake: 0.154057
Epoch [24/100], d_loss:

Epoch [43/100], d_loss: 0.433510, g_loss: 2.702932 D real: 0.856518, D fake: 0.095435
Epoch [43/100], d_loss: 0.509821, g_loss: 2.366928 D real: 0.810647, D fake: 0.063912
Epoch [43/100], d_loss: 0.333256, g_loss: 3.529268 D real: 0.874628, D fake: 0.056821
Epoch[43/100], time_consumed:36.695875
Epoch [44/100], d_loss: 0.459834, g_loss: 1.609287 D real: 0.827810, D fake: 0.099070
Epoch [44/100], d_loss: 0.355865, g_loss: 3.089469 D real: 0.866868, D fake: 0.086688
Epoch [44/100], d_loss: 0.435350, g_loss: 3.246578 D real: 0.913775, D fake: 0.190199
Epoch [44/100], d_loss: 0.349217, g_loss: 2.432211 D real: 0.893850, D fake: 0.125404
Epoch[44/100], time_consumed:36.924956
Epoch [45/100], d_loss: 0.392063, g_loss: 3.492782 D real: 0.848076, D fake: 0.055623
Epoch [45/100], d_loss: 0.423357, g_loss: 2.527925 D real: 0.855045, D fake: 0.128001
Epoch [45/100], d_loss: 0.426341, g_loss: 2.715391 D real: 0.856482, D fake: 0.122403
Epoch [45/100], d_loss: 0.622256, g_loss: 3.557329 D real: 0.7

Epoch [64/100], d_loss: 0.337083, g_loss: 3.581308 D real: 0.907106, D fake: 0.118184
Epoch[64/100], time_consumed:36.347556
Epoch [65/100], d_loss: 0.374472, g_loss: 4.049211 D real: 0.872655, D fake: 0.059659
Epoch [65/100], d_loss: 0.309309, g_loss: 2.496125 D real: 0.898606, D fake: 0.102688
Epoch [65/100], d_loss: 0.347196, g_loss: 3.223670 D real: 0.927699, D fake: 0.149783
Epoch [65/100], d_loss: 0.321769, g_loss: 3.114597 D real: 0.881916, D fake: 0.066611
Epoch[65/100], time_consumed:36.351794
Epoch [66/100], d_loss: 0.248770, g_loss: 2.500632 D real: 0.939085, D fake: 0.114894
Epoch [66/100], d_loss: 0.348926, g_loss: 3.021822 D real: 0.924568, D fake: 0.135701
Epoch [66/100], d_loss: 0.293401, g_loss: 3.262565 D real: 0.926411, D fake: 0.102385
Epoch [66/100], d_loss: 0.313752, g_loss: 2.922969 D real: 0.897924, D fake: 0.072163
Epoch[66/100], time_consumed:36.801135
Epoch [67/100], d_loss: 0.230784, g_loss: 2.885333 D real: 0.938537, D fake: 0.113499
Epoch [67/100], d_loss:

Epoch [86/100], d_loss: 0.268012, g_loss: 3.863236 D real: 0.916094, D fake: 0.064627
Epoch [86/100], d_loss: 0.226572, g_loss: 3.128134 D real: 0.913640, D fake: 0.056674
Epoch [86/100], d_loss: 0.249994, g_loss: 2.720984 D real: 0.948540, D fake: 0.122466
Epoch[86/100], time_consumed:45.698966
Epoch [87/100], d_loss: 0.186213, g_loss: 2.991390 D real: 0.947225, D fake: 0.089343
Epoch [87/100], d_loss: 0.214101, g_loss: 4.987540 D real: 0.905224, D fake: 0.020228
Epoch [87/100], d_loss: 0.378494, g_loss: 3.103568 D real: 0.941872, D fake: 0.196948
Epoch [87/100], d_loss: 0.272523, g_loss: 4.591593 D real: 0.915159, D fake: 0.054738
Epoch[87/100], time_consumed:46.126366
Epoch [88/100], d_loss: 0.245874, g_loss: 3.150447 D real: 0.924492, D fake: 0.084788
Epoch [88/100], d_loss: 0.295967, g_loss: 3.548120 D real: 0.896674, D fake: 0.043052
Epoch [88/100], d_loss: 0.270973, g_loss: 2.849651 D real: 0.920404, D fake: 0.092980
Epoch [88/100], d_loss: 0.302595, g_loss: 3.048847 D real: 0.9